In [1]:
import random
import copy as cp
import itertools
import gym
import numpy as np
from gym import spaces
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.feature_extraction import FeatureHasher
import pandas as pd

import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

### Env:

In [2]:
### ENVIRONMENT from OpenAI
class SplendorEnv(gym.Env):
    MAX_GEMS_SUPPLY=10 #general attribute to call and compare againtst gems
    MAX_JOKERS=3
    def __init__(self, card_supply, deck_nobles, initial_gems =[5,5, 5, 5, 5,3], cards_player=None, nobles=None):
        '''Let's define the initial state and variables.'''
        num_card_tiers = 3
        cards_per_tier = 4
        num_token_colors = 5
        num_nobles = 3
        ##Variable to follow the number of turns:
        self.turns=0
        #creates the card decks
        deck=cp.deepcopy(card_supply) #since we are using pop we need to define a new list removing references. 
        self.deck_1 = deck[0]
        self.deck_2 = deck[1]
        self.deck_3 = deck[2]

        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()

        #creates the noble board
        self.nobles = []
        self.CreateNobles(deck_nobles)

        #Create the gems
        self.gem_reserve= initial_gems ## SUPPLY GEMS
        self.gems = [0,0,0,0,0,0] ##AGENT GEMS

        #create the gems combinations
        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))
        
        
        ##Valid actions:
        self.all_actions = []
        self.create_all_actions()

        self.valid_actions=[0]*len(self.all_actions)

        self.score=0

        ##For our agent:
        self.player_cards = cards_player if cards_player is not None else [] # cars in the player power
        self.player_reserved_cards = [] #cards reserved by the player
        self.nobles = nobles if nobles is not None else [] #nobles the player has
        self.buying_power = self.gems # buying power is initializes equal to the amount of gems of each kind

        ##Binary list for combinations of gems:
        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))

        ##Simpler version:
        self.observation_space = gym.spaces.Dict({
            'player_gems': spaces.Box(low=0, high=7, shape=(num_token_colors+1,), dtype=np.integer), #array 6 elements
            'gems_supply': spaces.Box(low=0, high=7, shape=(num_token_colors+1,), dtype=np.integer), #array 6
            'player_cards': spaces.Box(low=0, high=np.inf, shape=(num_token_colors,), dtype=np.integer), #array 5
            'player_score': spaces.Discrete(30), # integer
            'nobles': spaces.Tuple(tuple([spaces.Tuple((spaces.MultiDiscrete([1,1,1,1,1]), spaces.Discrete(5))) for _ in range(num_nobles)])), #cost and points per noble
            'player_reserved':spaces.Discrete(2),
            #'Board': spaces.Box(low=0, high=np.inf, shape=(3, 4), dtype=np.integer),  # Add the 3x3 board here
            'valid_actions':spaces.Discrete(57)
            })
        
        self.action_space = spaces.Dict({
            'action_type': spaces.Discrete(4),  # Action type (0-3)
            'purchase_card': spaces.Tuple((
                spaces.Discrete(num_card_tiers),    # Card tier (0-2)
                spaces.Discrete(cards_per_tier),    # Card index (0-cards_per_tier)
            )),
            'reserve_card': spaces.Tuple((
                spaces.Discrete(num_card_tiers),    # Card tier (0-2)
                spaces.Discrete(cards_per_tier),    # Card index (0-cards_per_tier)
            )),
            'buy_reserved': spaces.Discrete(3),  # Action type (0-3)
            'pick_tokens': spaces.MultiBinary(num_token_colors)  # Tokens to pick
        })

   
    def UpdateCardBoard(self):
        '''Randomly shuffle each deck and take the first 3 cards to place them on the board
        Updates boards to 3 if there are less than 3 cards'''
        random.shuffle(self.deck_1)
        random.shuffle(self.deck_2)
        random.shuffle(self.deck_3)
        #update cards rank 1
        while len(self.board_1) <4:
            if len(self.deck_1) ==0:
                break
            self.board_1.append(self.deck_1.pop(0))
            
        #update cards rank 2
        while len(self.board_2) <4:
            if len(self.deck_2) ==0:
                break
            self.board_2.append(self.deck_2.pop(0))
            
        #update cards rank 3
        while len(self.board_3) <4:
            if len(self.deck_3) ==0:
                break
            self.board_3.append(self.deck_3.pop(0))

    def create_all_actions(self):
        '''
        create all the actions possible
         '''
        #all the buying actions possible

        card_buy = []
        for board in [self.board_1, self.board_2, self.board_3]:   
                for card in board:
                    
                    card_buy.append((card.level-1,board.index(card))) # we append a tuple containing level(0-2),index in board(0-3)
        self.all_actions.extend(  [('buy card', x) for x in card_buy])

        #all the reseravations possible possible
        reserve_cards = []
        for board in [self.board_1, self.board_2, self.board_3]:
                for card in board:
                            reserve_cards.append((card.level-1,board.index(card)))
        self.all_actions.extend( [('reserve', x) for x in reserve_cards])  

        #buy all the cards possible in our reserved cards
        for i in range(self.MAX_JOKERS):
             self.all_actions.append(('buy reserved', i))
        
        #append all the possible token combinations
        self.all_actions.extend([('take', x) for x in self.binary_lists_with_sum_of_2])
        self.all_actions.extend([('take', x) for x in self.binary_lists_with_sum_of_3])
        self.all_actions.extend([('take', x) for x in [[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1], [2,0,0,0,0], [0,2,0,0,0], [0,0,2,0,0], [0,0,0,2,0], [0,0,0,0,2]]])
        #last resort actions
        negative_gems = [[ -x for x in inner_list] for inner_list in self.binary_lists_with_sum_of_3 ]
        self.all_actions.extend([('take',x) for x in negative_gems ])     

    
    
    def CreateNobles(self, deck_nobles):
        '''Shuffle the deck of Nobles and get the first 3 cards - no replacement here.'''
        deck_n=cp.deepcopy(deck_nobles)
        random.shuffle(deck_n)

        #update cards rank 1
        while len(self.nobles) <4:
            self.nobles.append(deck_n.pop(0)) #the issue for reseting is that when we pop we modify the original
    
    def UpdateBuyingPower(self): 
        ''' Buying power is gems + cards bought of each color'''
        self.buying_power =cp.deepcopy(self.gems)
        for card in self.player_cards:
            self.buying_power[card.color] +=1 # if you have bought a card of said color you get an extra buying unit for that color.

    def get_legal_actions(self):
        '''We get all legal actions as indexes:
        Buy Card , (i,j) #specifies which card to buy
        Reserve Card, (i,j) #card to reserve
        Buy reserved (i) # index in reserved cards.
        Take Gems, [x,x,x,x,x] #gems to take
        ''' 
        #

        ##First we get all legal actions:
        legal_actions=[] #57 available actions:
        ##Check cards that can be bought based on updated buying power'''
        card_buy = self.get_purchasable_cards() #12
        legal_actions.extend(card_buy)
        ##Now reservable card(should be all as long as we have enough space and gems)'''
        reserve_cards=self.get_reservable_cards() #12
        legal_actions.extend(reserve_cards)
        ##We get the purchasable reserved cards:
        buyable_reserved_cards=self.get_purchasable_reserved()
        legal_actions.extend(buyable_reserved_cards)
        ##Takeable gems combinations:
        gem_buy=self.get_takable_gems()
        legal_actions.extend(gem_buy)
        #return self.transform_legal_actions(legal_actions) #to transform list of tuples to dictionary of 4 main actions.
        if len(legal_actions)==0:
             last_resort_action=[]
             for gem in self.gems[:5]:
                  if gem>0 and sum(last_resort_action) > -3:#we remove 1 gem of each gem reserve with more than 0 
                       last_resort_action.append(-1)
                  else:
                       last_resort_action.append(0)
             legal_actions=[('take',last_resort_action)]
        
        self.valid_actions=[0]*len(self.all_actions)
        for x in legal_actions:
             self.valid_actions[self.all_actions.index(x)] = 1
        return legal_actions           
    
    
    def get_purchasable_cards(self):
        '''We get all purchasable cards based on buying power(gems + cards) and return the index (level, column) in list format'''
        card_buy=[]
        for board in [self.board_1, self.board_2, self.board_3]:   
                for card in board:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])
                    if over <= self.gems[-1]:
                        card_buy.append((card.level-1,board.index(card))) # we append a tuple containing level(0-2),index in board(0-3)
        return  [('buy card', x) for x in card_buy]

    def get_reservable_cards(self):
        '''Check reservable cards based on number of gems and number of reserved cards.'''
        reserve_cards = []
        if sum(self.gems) <10 and len(self.player_reserved_cards) < self.MAX_JOKERS:
            for board in [self.board_1, self.board_2, self.board_3]:
                    for card in board:
                            reserve_cards.append((card.level-1,board.index(card)))
        return [('reserve', x) for x in reserve_cards]         
        
    def get_purchasable_reserved(self):
        '''Checks fi we can buy our reserved cards and return index of reserved card purchasable'''
        buyable_reserved_cards = []
        for card in self.player_reserved_cards:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])

                    if over <= self.gems[-1]:
                        buyable_reserved_cards.append(self.player_reserved_cards.index(card)) # returns index in reserved cards 0-2
        return [('buy reserved', x) for x in buyable_reserved_cards]

    def get_takable_gems(self):
        '''We check all possible combinations of gem taking doubles and triples.'''
        #make the list of gems currently takable
        doable_3gems = [] #all possible combinations for 3 gems
        for comb in self.binary_lists_with_sum_of_3:
            if  min([self.gem_reserve[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_3gems.append(comb)

        doable_2gems = [] #possible combinations for 2 gems (same color)
        for comb in self.binary_lists_with_sum_of_2:
            if  min([self.gem_reserve[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_2gems.append(comb)    

        doable_1gems = [] #possible combinations for 2 gems (same color)
        for comb in [[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]:
            if  min([self.gem_reserve[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_1gems.append(comb)      
        ##max 10 combinations for 2 and 3 so 20 in total.
        gem_buy= []

        # If gem supply is 10, don't consider taking gems
        if sum(self.gems) ==SplendorEnv.MAX_GEMS_SUPPLY:
            return [] # don't return gems to take.
        
        #append the list of 3 gems if the player has already 7 gems or less
        if sum(self.gems) <= 7:
            gem_buy.extend(doable_3gems)

        #append the list of 2 gems if the player has already 8

        if sum(self.gems) == 8:
            gem_buy.extend(doable_2gems)
        
        # append the list of 1 gem if the player has already 9
        
        if sum(self.gems) == 9:
            gem_buy.extend(doable_1gems)

        #Appends the duos
        ##If the supply of gems is greater than 3 we can alwasy take 2 of the same gems.
        if sum(self.gems)<=8:
            for x in range(len(self.gems[:5])):
                    list_empty = [0,0,0,0,0]
                    if self.gem_reserve[x] >3:
                        list_empty[x] = 2
                        if list_empty != [0,0,0,0,0]:
                            gem_buy.append(list_empty)

        return [('take', x) for x in gem_buy]

    
    def execute_action(self, action): 
        '''Once an actions has been chosen we execute it by calling the action function'''

        # execute the specified action and update the state
        action_type, action_params = action
        if action_type == 'buy card':
            index = action_params
            self.buy_card(index)

        elif action_type == 'buy reserved':
            index = action_params
            self.buy_reserved_card(index)

        elif action_type == 'reserve':
            index = action_params
            self.reserve_card(index)

        elif action_type == 'take':
            gems = action_params
            self.take_gems(gems)
    

    def buy_card(self, index):
        '''We buy the selected card and update the environment by removing the bought card from the board, updating player's gems and supply gems'''
        # update gem and card supply and player's state when a card is bought
        # we get the card from the index:
        card=None
 
        while card is None:
            for board in [self.board_1, self.board_2, self.board_3]:
                        for cards in board:
                             if index == (cards.level-1,board.index(cards)):
                                  card=cards
                                
        #Before adding card to players hand we need to check how much buying power do our current cards give us:
        #extract the colors of the current cards
        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        #add card to players cards
        self.player_cards.append(card)

        ##Now we remove it from the board
        if card.level ==1:
             self.board_1.remove(card)
        
        elif card.level ==2:
             self.board_2.remove(card)

        elif card.level ==1:
             self.board_3.remove(card)

        ##First check from the cost we substract our current free buying power.
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        #print('Discounted card cost:',new_cost)


        #compute required tokens
        over = 0
     
        '''I'm using gem_reserve instead of buying power, I think it will be better to keep the values from gem reserve and cards separated to know what gems to resupply and which not'''
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers

            over = max(0, new_cost[i]- self.gems[i]) #jokers needed for this color
            gem_transaction = max(0, new_cost[i] - over)# after substracting cards + joker we get the amount to pay.
            if over>self.gems[5]:
                 #print(over,self.gem_reserve)
                 raise ValueError("Too many jokers")
            
            #print('gem_transaction?',gem_transaction)
            self.gems[i] -= gem_transaction
            self.gem_reserve[i]  += gem_transaction
            self.gems[5]-=over
            self.gem_reserve[5]+=over

    def reserve_card(self, index):
        '''We reserve the selected card and remove it from the board'''
        ##Get the card:
        card=None
        while card is None:
            
            for board in [self.board_1, self.board_2, self.board_3]:
                        for cards in board:
                             if index == (cards.level-1,board.index(cards)):
                                  card=cards
        #add card to players reserved cards
        self.player_reserved_cards.append(card)

        #get rid of the card on the board
        if card.level ==1:
             self.board_1.remove(card)
        
        elif card.level ==2:
             self.board_2.remove(card)

        elif card.level ==3:
             self.board_3.remove(card)

        #get a joker
        self.gems[5] += 1 # add joker to hand.
        self.gem_reserve[5]-=1 # added removal of joker from board

    def buy_reserved_card(self, index):
        '''We buy the selected reserved card and update the environment'''
        ##Get reserved card:
        #take the card out of the reserved cards
        
        card=self.player_reserved_cards[index]
        self.player_reserved_cards.remove(card)

        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        # update gem and card supply and player's state when a card is bought
        #add card to players cards
        self.player_cards.append(card)

        #compute required tokens
        over = 0
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers
            over=max(0,new_cost[i]-self.gems[i])
            #update the number of reamining gems
            if over>self.gems[5]:
                 raise ValueError("Too many jokers needed")
            gem_transaction = max(0,new_cost[i]-over)
            self.gems[i] -=  gem_transaction
            self.gem_reserve[i] += gem_transaction
            self.gem_reserve[5]+=over
            self.gems[5]-=over

    def take_gems(self, gems_array):
        '''Add gems from supply to player's reserve and remove it from supply.'''

        # simply add the gems to the players count
        for i in range(len(gems_array)):
            self.gems[i] += gems_array[i]
            self.gem_reserve[i] -= gems_array[i]


    def step(self, action):
        
        self.execute_action(action)
        self.turns+=1
        self.UpdateBuyingPower()
        self.UpdateCardBoard()

        # Calculate reward
        reward,_ ,done = self.calculate_reward()
        if action[0]=='take' and sum(action[1])<0:
             reward-=5
        #We compute the new legal actions for the next state to add to the next obs
        self.get_legal_actions()

        ##Update observation:
        next_obs = {
        'player_gems': self.gems,  # player has no gems initially
        'gems_supply': self.gem_reserve,  # 4 gems of each color except for gold, which has 5
        'player_cards': self.player_cards,  # player has no cards initially
        'player_score': self.score,  # player has no score initially
        'nobles':self.nobles, 
        'player_reserved':self.player_reserved_cards,
        'valid_actions':self.valid_actions,
        'Board': np.array([self.board_1, self.board_2, self.board_3], dtype=object),
        'all_actions': self.all_actions

        }
        # Return the next state, reward, done, and additional info
        return next_obs, reward, done, {}

    def calculate_reward(self, max_turns = 60):
        '''Count points and win condition'''
        reward=0
        points =  0
        win = False

        for cards in self.player_cards:
              points +=cards.points  

        for noble in self.nobles:
             points +=noble[1]

        if points >= 15:
             win = True
             reward = max_turns - self.turns

        if self.turns > max_turns:
             win= True
             reward = -50
        self.score = points

        return reward,points, win

    def reset(self, new_deck,new_deck_nobles, reset_gems = [4,4,4,4,4,3]):
        '''Reset function, re-initializes all environmental variables:
        Set score to 0, reset supply, decks, boards, reserved cards, nobles etc.'''
        ###Reset all variables and rebuild:
        ##Reset turns:
        self.turns=0
        ##Decks:
        self.deck_1=[]
        self.deck_2=[]
        self.deck_3=[]

        #creates the card decks
        self.deck_1 = cp.deepcopy(new_deck[0])
        self.deck_2 = cp.deepcopy(new_deck[1])
        self.deck_3 = cp.deepcopy(new_deck[2])
        ##Boards:
        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()
        ##Nobles:
        #creates the noble board
        self.nobles = []
        self.CreateNobles(new_deck_nobles)
        ##Gems:
        #Create the gems
        self.gems=[0,0,0,0,0,0]
        ##Agentwise:
        self.score=0
        # Reset the agent's variables
        ##For our agent:
        self.gem_reserve = [4,4,4,4,4,3] # gems in the agents hand initializes as 0
        self.player_cards = [] # cars in the player power
        self.player_reserved_cards = [] #cards reserved by the player
        self.nobles = [] #nobles the player has
        self.buying_power = self.gems # buying power is initializes equal to the amount of gems of each kind

        
        ##Update legal actions:
        self.get_legal_actions()

        ###OBS:
        obs = {
        'player_gems': self.gems,  # player has no gems initially
        'gems_supply': self.gem_reserve,  # 4 gems of each color except for gold, which has 5
        'player_cards': self.player_cards,  # player has no cards initially
        'player_score': self.score,  # player has no score initially
        'nobles':self.nobles, 
        'player_reserved':self.player_reserved_cards,
        'valid_actions':self.valid_actions,
        'Board': np.array([self.board_1, self.board_2, self.board_3], dtype=object),
        'all_actions': self.all_actions
        }

        return obs

    def render(self, mode='human'):
        pass

    def close(self):
        pass


In [3]:
class Card:
    def __init__(self, level, color, cost, points):
        '''Cards are defined by level color cost and points they give'''
        #int ranges from 1 to 3
        self.level = level
        #We would keep as int based on this mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}(alphabetical order)
        self.color = color
        #We do cost as an array [black, ...., white] [0, 2, 1, 0, 0]
        self.cost = cost
        #int, ranges from 0 to 5
        self.points = points

class Noble:
    def __init__(self, requirements, points):
        '''For nobles we only care about requirements and points you get'''
        #We do requirements as an array [black, ...., white] [3, 3, 3, 0, 0]
        self.requirements = requirements
        #int ranges from 3 to 4
        self.points = points

class GemColor:
    def __init__(self, name):
        self.name = name

### Import cards:

In [4]:
#mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}

cards_1 = []
cards_2 = []
cards_3 = []

cards = pd.read_csv('card_data.csv').fillna(0)
for index, row in cards.iterrows():
    #get cost
    cost = [row['cost_black'], row['cost_blue'], row['cost_green'], row['cost_red'], row['cost_white']]
    cost =  list(map(int, cost))
    #map to get color
    color_dict = {'black':0, 'blue' :1, 'green':2, 'red': 3, 'white': 4}
    color = color_dict[row['color']]

    card = Card(row['tier'], color, cost, row['points'])

    if row['tier'] ==1:
        cards_1.append(card)

    elif row['tier'] ==2:
        cards_2.append(card)

    else :
        cards_3.append(card)

# create nobles
nobles= [Noble([3,3,0,0,3], 3),
Noble([3,0,3,3,0], 3),
Noble([0,4,0,4,0], 3),
Noble([0,0,4,0,4], 3),
Noble([4,0,0,0,4], 3),
Noble([0,0,0,5,3], 3)]

# create SplendorState
'''Note, card supply and nobles are list of lists of lengths 3 and 1 respectively.'''
##Get the card supply:
card_supply = [cards_1, cards_2, cards_3]

In [5]:
def render_state(state):
    # Print player's gem count and reserve cards
    print(f"Player gems: {state['player_gems']}")
    print("Reserved cards:")
    for card in state['player_reserved']:
        print(f"  Level {card.level} {card.color} card worth {card.points} points, cost: {card.cost}")

    # Print board and player's purchased cards
    print("Board and player's purchased cards:")
    print("Level | Black | Blue | Green | Red | White | Points |")
    print("---------------------------------------------------")
    for i in range(3):
        row = f"  {i+1}   |"
        for j in range(4):
            card = state['Board'][i][j]
            if card is None:
                row += "      |"
            else:
                row += f" {card.color}:{card.cost} |"
        row += "       |"
        print(row)

    print("Player's purchased cards:")
    for card in state['player_cards']:
        print(f"  Level {card.level} {card.color} card worth {card.points} points, cost: {card.cost}")

    # Print nobles
    print("Nobles:")
    for noble in state['nobles']:
        print(f"  Requirements: {noble.requirements}, Points: {noble.points}")

### Random Agent

In [13]:
import gym

env = SplendorEnv(card_supply, nobles)
obs = env.reset(card_supply, nobles)
done = False
actions_list=[]

while not done:
    list_actions =[]
    for x in range(len(obs['valid_actions'])):
        if obs['valid_actions'][x] ==1:
            list_actions.append(obs['all_actions'][x])
    random_action=random.choice(list_actions)   
    actions_list.append(random_action)
    n_obs,reward,done,_= env.step(random_action)
    obs=n_obs
    render_state(obs)

print(f'Fernando made {env.score} points in {env.turns} turns, by making the following actions {actions_list}' )
env.close()


Player gems: [0, 1, 1, 1, 0, 0]
Reserved cards:
Board and player's purchased cards:
Level | Black | Blue | Green | Red | White | Points |
---------------------------------------------------
  1   | 3:[0, 0, 0, 2, 2] | 4:[2, 2, 0, 0, 0] | 1:[1, 0, 1, 1, 1] | 1:[2, 0, 0, 0, 1] |       |
  2   | 4:[2, 0, 1, 4, 0] | 2:[2, 3, 0, 0, 2] | 1:[3, 2, 3, 0, 0] | 1:[0, 6, 0, 0, 0] |       |
  3   | 3:[0, 0, 7, 3, 0] | 3:[3, 5, 3, 0, 3] | 4:[3, 3, 3, 5, 0] | 4:[7, 0, 0, 0, 0] |       |
Player's purchased cards:
Nobles:
Player gems: [0, 1, 1, 1, 0, 1]
Reserved cards:
  Level 1 1 card worth 0 points, cost: [2, 0, 0, 0, 1]
Board and player's purchased cards:
Level | Black | Blue | Green | Red | White | Points |
---------------------------------------------------
  1   | 3:[0, 0, 0, 2, 2] | 4:[2, 2, 0, 0, 0] | 1:[1, 0, 1, 1, 1] | 4:[1, 1, 0, 0, 3] |       |
  2   | 4:[2, 0, 1, 4, 0] | 2:[2, 3, 0, 0, 2] | 1:[3, 2, 3, 0, 0] | 1:[0, 6, 0, 0, 0] |       |
  3   | 3:[0, 0, 7, 3, 0] | 3:[3, 5, 3, 0, 3] | 4:[

c:\Users\CompuTop\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\spaces\box.py:84: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  self.dtype = np.dtype(dtype)


In [15]:
import gym

episodes=100
turns=[]
for x in range(episodes):
    env = SplendorEnv(card_supply, nobles)
    obs = env.reset(card_supply, nobles)
    done = False
    actions_list=[]
    while not done:
        #random_action = random.choice(obs['valid_actions'])
        list_actions =[]
        for x in range(len(obs['valid_actions'])):
            if obs['valid_actions'][x] ==1:
                list_actions.append(obs['all_actions'][x])
        random_action=random.choice(list_actions)   
        actions_list.append(random_action)
        n_obs,reward,done,_= env.step(random_action)
        obs=n_obs
        #env.render()
    turns.append(env.turns)
    env.close()
    
average_turns = sum(turns) / len(turns)
print("Average score over {} episodes: {:.2f}".format(episodes,average_turns))
losses=turns.count(61) ## Number of losses.
print('Number of Losses:{}'.format(losses))
filtered_scores = [score for score in turns if score != 61]
print('Average turns to win without losses:',sum(filtered_scores) / len(filtered_scores))

Average score over 100 episodes: 41.63
Number of Losses:0
Average turns to win without losses: 41.63


### Visualization of Board

In [8]:
# Define constants for colors and card dimensions
import pygame

CARD_WIDTH = 40
CARD_HEIGHT = 60
CARD_PADDING = 10
CARD_FONT_SIZE = 10
COLORS = {
    0: (0, 0, 0),  # black
    1: (0, 0, 255),  # blue
    2: (0, 255, 0),  # green
    3: (255, 0, 0),  # red
    4: (255, 255, 255),  # white
    5: (255, 255, 0),  # yellow
}

def draw_card(screen, card, x, y):
    # Draw the card background
    pygame.draw.rect(screen, COLORS[card.color], (x, y, CARD_WIDTH, CARD_HEIGHT))

    # Draw the point label
    point_label = pygame.font.SysFont(None, CARD_FONT_SIZE).render(str(card.points), True, (0, 0, 0))
    if card.color == 0:
        point_label = pygame.font.SysFont(None, CARD_FONT_SIZE).render(str(card.points), True, (255, 255, 255))
    screen.blit(point_label, (x + CARD_PADDING, y + CARD_PADDING))

    # Draw the cost squares
    cost_x = x + CARD_WIDTH - CARD_PADDING - CARD_FONT_SIZE
    cost_y = y + CARD_PADDING
    for i, cost in enumerate(card.cost):
        cost_color = COLORS[i]
        cost_rect = pygame.Rect(cost_x, cost_y, CARD_FONT_SIZE, CARD_FONT_SIZE)
        
        pygame.draw.rect(screen, cost_color, cost_rect)
        cost_label = pygame.font.SysFont(None, CARD_FONT_SIZE).render(str(cost), True, (255, 255, 255))
        if i == 4:
            cost_label = pygame.font.SysFont(None, CARD_FONT_SIZE).render(str(cost), True, (0, 0, 0))
        cost_label_rect = cost_label.get_rect(center=cost_rect.center)
        screen.blit(cost_label, cost_label_rect)

        cost_y += CARD_FONT_SIZE + CARD_PADDING

# Define function to render the state
def render_state(state):
    # Initialize Pygame
    pygame.init()

    # Create the screen
    screen_width = 6 * CARD_WIDTH + 7 * CARD_PADDING
    screen_height = 6 * CARD_HEIGHT + 7 * CARD_PADDING + CARD_FONT_SIZE * 2
    screen = pygame.display.set_mode((screen_width, screen_height))

    # Draw the board cards
    for i in range(3):
        for j in range(4):
            card = state['Board'][i][j]
            if card is not None:
                draw_card(screen, card, j * (CARD_WIDTH + CARD_PADDING) + CARD_PADDING, i * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*3 + CARD_FONT_SIZE * 2)

    # Draw a separator line between the board and the player's cards
    pygame.draw.line(screen, (255, 255, 255), (0, 3 * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*3 + CARD_FONT_SIZE * 2), (screen_width, 3 * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*3 + CARD_FONT_SIZE * 2), 2)

    # Draw the player's purchased cards
    for i, card in enumerate(state['player_cards']):
        draw_card(screen, card, i * (CARD_WIDTH + CARD_PADDING) + CARD_PADDING, 3 * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*10 + CARD_FONT_SIZE * 2)

    # Draw labels for the board and player's cards sections
    board_label = pygame.font.SysFont(None, CARD_FONT_SIZE+10).render("Board", True, (255, 255, 255))
    screen.blit(board_label, (0, CARD_PADDING*3))
    player_cards_label = pygame.font.SysFont(None, CARD_FONT_SIZE +10).render("Player's Cards (first 5)", True, (255, 255, 255))
    screen.blit(player_cards_label, (0, 3 * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*8))

    # Draw the cost squares
    gem_x = CARD_WIDTH*2
    gem_y = 3 * (CARD_HEIGHT + CARD_PADDING) + CARD_PADDING*8
    for i, gem in enumerate(state['player_gems']):
        cost_color = COLORS[i]
        cost_rect = pygame.Rect(gem_x, gem_y, CARD_FONT_SIZE*2, CARD_FONT_SIZE*2)
        
        pygame.draw.rect(screen, cost_color, cost_rect)
        gem_label = pygame.font.SysFont(None, CARD_FONT_SIZE*2).render(str(gem), True, (255, 255, 255))
        if i == 4:
            gem_label = pygame.font.SysFont(None, CARD_FONT_SIZE*2).render(str(gem), True, (0, 0, 0))
        gem_label_rect = gem_label.get_rect(center=cost_rect.center)
        screen.blit(gem_label, gem_label_rect)

        gem_x += CARD_FONT_SIZE*3

   

    # Update the screen
    pygame.display.flip()

In [9]:
import gym

env = SplendorEnv(card_supply, nobles)
obs = env.reset(card_supply, nobles)
done = False
actions_list=[]
while not done:
    list_actions =[]
    for x in range(len(obs['valid_actions'])):
        if obs['valid_actions'][x] ==1:
            list_actions.append(obs['all_actions'][x])
    random_action=random.choice(list_actions)   
    actions_list.append(random_action)
    n_obs,reward,done,_= env.step(random_action)
    obs=n_obs

render_state(obs)
print(f'Fernando made {env.score} points in {env.turns} turns, by making the following actions {actions_list}' )
env.close()

: 

### Pre-processing functions:

In [6]:
def flatten_state(state):
    flat_state = {}
    
    for idx, value in enumerate(state['player_gems']):
        flat_state[f'player_gems_{idx}'] = value

    for idx, value in enumerate(state['gems_supply']):
        flat_state[f'gems_supply_{idx}'] = value

    for idx, rcard in enumerate(state['player_reserved']):
        flat_state[f'player_reserved_{idx}_level'] = rcard.level
        flat_state[f'player_reserved_{idx}_color'] = rcard.color
        flat_state[f'player_reserved_{idx}_points'] = rcard.points
        for jdx, cost in enumerate(rcard.cost):
            flat_state[f'player_reserved_{idx}_cost_{jdx}'] = cost
        
    flat_state['player_score'] = state['player_score']

    for idx, noble in enumerate(state['nobles']):
        for jdx, req in enumerate(noble.requirements):
            flat_state[f'noble_{idx}_req_{jdx}'] = req
        flat_state[f'noble_{idx}_points'] = noble.points

    for idx, card in enumerate(state['player_cards']):
        flat_state[f'player_card_{idx}_level'] = card.level
        flat_state[f'player_card_{idx}_color'] = card.color
        flat_state[f'player_card_{idx}_points'] = card.points
        for jdx, cost in enumerate(card.cost):
            flat_state[f'player_card_{idx}_cost_{jdx}'] = cost
    
    for i in range(state['Board'].shape[0]):
        for j in range(len(state['Board'][0])):
            card = state['Board'][i][j]
            idx = i*4 + j
            flat_state[f'Board_{idx}_level'] = card.level
            flat_state[f'Board_{idx}_color'] = card.color
            flat_state[f'Board_{idx}_points'] = card.points
            for jdx, cost in enumerate(card.cost):
                flat_state[f'Board_{idx}_cost_{jdx}'] = cost

    return flat_state

## DQN Architecture

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 0.1 # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.model = self._build_model()

    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, valid_actions, all_actions,train=True):
        if train==False:
            self.epsilon=0
        if np.random.rand() <= self.epsilon:
            y=0
            if self.epsilon > self.epsilon_min:
               self.epsilon*=self.epsilon_decay
            while y==0:
                cand=random.choice(all_actions)
                y=valid_actions[all_actions.index(cand)]
            return cand
        
        act_values = self.model.predict(state, verbose = 0)[0]
        index_list = 0
        max_value = -np.inf
        for x in range(len(act_values)):
            if (act_values[x] >max_value) and (valid_actions[x]==1):
                index_list  = x
                max_value = act_values[x]
        return all_actions[index_list]
    
    
    def update_reward_for_failed_game(self, new_reward=-180):
        '''Function to update rewards of steps in failed game to -100'''
        for i in range(-60, 0):
            state, action, _, next_state, done = self.memory[i]
            self.memory[i] = (state, action, new_reward/(-i), next_state, done) #update reward more negative the later in the game


    def replay(self, batch_size,replay_failed_game=False):
        if replay_failed_game:
            ##we replay the game we didn't win:
            self.update_reward_for_failed_game() 
            minibatch = list(self.memory)[-60:]
        else:
            minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state,verbose = 0)[0])
            target_f = self.model.predict(state, verbose = 0)
            print(target_f.shape)
            target_f[0][env.all_actions.index(action)] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


## DQN TORCH IMPLEMENTATION RUNS MUCH FASTER WITH GPU

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

class DQNNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    # def __init__(self, state_size, action_size):
    #     super(DQNNetwork, self).__init__()
    #     self.fc1 = nn.Linear(state_size, 64)
    #     self.fc2 = nn.Linear(64, 128)
    #     self.fc3 = nn.Linear(128, 64)
    #     self.fc4 = nn.Linear(64, 32) # Add another fully connected layer
    #     self.fc5 = nn.Linear(32, action_size) # Adjust the output size accordingly
    #     self.dropout = nn.Dropout(p=0.25)

    # def forward(self, x):
    #     x = torch.relu(self.fc1(x))
    #     x = self.dropout(x)
    #     x = torch.relu(self.fc2(x))
    #     x = torch.relu(self.fc3(x))
    #     x = torch.relu(self.fc4(x)) # Add relu activation to the new layer
    #     x = self.fc5(x)
    #     return x


class DQNAgent:
    def __init__(self, state_size, action_size,device,gamma=0.95,epsilon=0.1,epsilon_min=0.01,epsilon_decay=0.995,learning_rate=0.0001):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = gamma  # discount rate
        self.epsilon = epsilon  # exploration rate
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.model = DQNNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.loss_function = nn.MSELoss() 
        self.device=device
        self.model.to(self.device)


    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, valid_actions, all_actions, train=True):
        if train == False:
            self.epsilon = 0
        if np.random.rand() <= self.epsilon:
            y = 0
            while y == 0:
                cand = random.choice(all_actions)
                y = valid_actions[all_actions.index(cand)]
            return cand
    
        state_tensor = state.clone().detach().unsqueeze(0)
        with torch.no_grad():
            act_values = self.model(state_tensor).cpu().numpy().flatten()
    
        index_list = 0
        max_value = -np.inf
        for x in range(len(act_values)):
            if (act_values[x] > max_value) and (valid_actions[x] == 1):
                index_list = x
                max_value = act_values[x]
    
        return all_actions[index_list]


    def update_reward_for_failed_game(self, new_reward=-100):
        for i in range(-60, 0):
            state, action, _, next_state, done = self.memory[i]
            self.memory[i] = (state, action, new_reward*(-i), next_state, done)

    def replay(self, batch_size, replay_failed_game=False):
        if replay_failed_game:
            self.update_reward_for_failed_game()
            minibatch = list(self.memory)[-60:]
        else:
            minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            state_tensor = state.clone().detach().unsqueeze(0)
            next_state_tensor = next_state.clone().detach().unsqueeze(0)

            action_index = env.all_actions.index(action)


            if not done:
                target = reward + self.gamma * torch.max(self.model(next_state_tensor)).item()
            else:
                target = reward

            target_q_values = self.model(state_tensor).clone().detach()[0]
            target_q_values[0][action_index] = target

            self.optimizer.zero_grad()
            current_q_values = self.model(state_tensor)[0]
            loss = self.loss_function(current_q_values, target_q_values)
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def load(self, name):
        self.model.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.state_dict(), name)

#### Tuning:

In [14]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.0 MB/s eta 0:00:00


In this case, the search space is continuous, and there is no fixed number of combinations. Optuna uses a Tree-structured Parzen Estimator (TPE) sampler by default, which is a Bayesian optimization method. It doesn't try all possible combinations; instead, it intelligently samples the search space based on previous trial results to find the optimal hyperparameters.

However, you have specified n_trials=100 in the study.optimize() function, which means Optuna will try 100 different combinations of hyperparameters, sampled from the search space using the TPE sampler. Note that these combinations will not cover the entire search space but rather focus on the most promising areas based on previous trial results.

In [15]:
###Based only on training data:
import optuna
import torch

EPISODES = 100
N_FEATURES = 1500
hasher = FeatureHasher(n_features=N_FEATURES, input_type='dict')

def train_agent(trial, device):
    state_size = N_FEATURES
    action_size = 67

    # Sample hyperparameters from the trial
    gamma = trial.suggest_float("gamma", 0.9, 0.99)
    epsilon = trial.suggest_float("epsilon", 0.1, 1.0)
    epsilon_min = trial.suggest_float("epsilon_min", 0.01, 0.1)
    epsilon_decay = trial.suggest_float("epsilon_decay", 0.9, 0.99)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)

    agent = DQNAgent(state_size, action_size, device, gamma, epsilon, epsilon_min, epsilon_decay, learning_rate)
    batch_size = 32

    training_cum_rewards = 0
    training_turns_per_eps = []

    for e in range(EPISODES):
        state = env.reset(card_supply, nobles)
        flat_state = flatten_state(state)
        state_hashed = hasher.transform([flat_state]).toarray()

        done = False
        while not done:
            state_hashed_torch = torch.tensor(state_hashed, dtype=torch.float32).to(device)
            action = agent.act(state_hashed_torch, state['valid_actions'], state['all_actions'])
            next_state, reward, done, _ = env.step(action)
            #print('Board Shape',next_state['Board'].shape) # sometimes shape 3,0
            flat_next_state = flatten_state(next_state)
            next_state_hashed = hasher.transform([flat_next_state]).toarray()

            next_state_hashed_torch = torch.tensor(next_state_hashed, dtype=torch.float32).to(device)
            agent.remember(state_hashed_torch, action, reward, next_state_hashed_torch, done)
            state = next_state
            state_hashed = next_state_hashed
            if done:
                training_cum_rewards += env.score
                training_turns_per_eps.append(env.turns)

        if len(agent.memory) > batch_size:
            if env.turns > 70:
                agent.replay(batch_size)
            else:
                agent.replay(batch_size)

    average_turns = sum(training_turns_per_eps) / len(training_turns_per_eps)
    return average_turns

if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Define the Optuna study and optimize the objective function
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: train_agent(trial, device), n_trials=50)

    # Print the best hyperparameters
    print("Best hyperparameters:", study.best_params)


[I 2023-04-15 13:08:08,735] A new study created in memory with name: no-name-990e4f74-c206-4a92-8555-93888c750f32
[I 2023-04-15 13:08:33,239] Trial 0 finished with value: 55.91 and parameters: {'gamma': 0.9521427606423774, 'epsilon': 0.4516194466781346, 'epsilon_min': 0.037455102001601966, 'epsilon_decay': 0.9162823769540225, 'learning_rate': 0.0005464884165343783}. Best is trial 0 with value: 55.91.
[I 2023-04-15 13:08:44,571] Trial 1 finished with value: 48.28 and parameters: {'gamma': 0.9648792443870994, 'epsilon': 0.8025244355499204, 'epsilon_min': 0.023906689591205545, 'epsilon_decay': 0.9788456720631304, 'learning_rate': 0.00029857030207094124}. Best is trial 1 with value: 48.28.
[I 2023-04-15 13:08:57,716] Trial 2 finished with value: 57.97 and parameters: {'gamma': 0.9443269543187581, 'epsilon': 0.5156073473354367, 'epsilon_min': 0.06590330000733545, 'epsilon_decay': 0.9496512444602169, 'learning_rate': 0.0009827872492963127}. Best is trial 1 with value: 48.28.
[I 2023-04-15 13

Best hyperparameters: {'gamma': 0.9629649349753686, 'epsilon': 0.6615728628093441, 'epsilon_min': 0.07456080325233885, 'epsilon_decay': 0.9858049704008014, 'learning_rate': 1.0352951751790414e-06}


 #### Tuning based on testing data:

In [16]:
import optuna
import torch

EPISODES = 200
N_FEATURES = 100000
hasher = FeatureHasher(n_features=N_FEATURES, input_type='dict') # fixed size : [ ]

def test_agent(agent, device):
    test_episodes = 100
    test_turns = []

    for x in range(test_episodes):
        env = SplendorEnv(card_supply, nobles)
        state = env.reset(card_supply, nobles)
        flat_state = flatten_state(state)
        state_hashed = hasher.transform([flat_state]).toarray()
        done = False

        while not done:
            state_hashed_torch = torch.tensor(state_hashed, dtype=torch.float32).to(device)
            action = agent.act(state_hashed_torch, state['valid_actions'], state['all_actions'], train=False)
            next_state, reward, done, _ = env.step(action)
            flat_next_state = flatten_state(next_state)
            next_state_hashed = hasher.transform([flat_next_state]).toarray()
            state = next_state
            state_hashed = next_state_hashed

        test_turns.append(env.turns)

    average_turns = sum(test_turns) / len(test_turns)
    filtered_scores = [score for score in test_turns if score != 61]
    filtered_average = sum(filtered_scores) / len(filtered_scores)
    return filtered_average

def train_agent(trial, device):
    state_size = N_FEATURES
    action_size = 67

    # Sample hyperparameters from the trial
    gamma = trial.suggest_float("gamma", 0.9, 0.99)
    epsilon = trial.suggest_float("epsilon", 0.1, 1.0)
    epsilon_min = trial.suggest_float("epsilon_min", 0.01, 0.1)
    epsilon_decay = trial.suggest_float("epsilon_decay", 0.9, 0.99)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)

    agent = DQNAgent(state_size, action_size, device, gamma, epsilon, epsilon_min, epsilon_decay, learning_rate)
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset(card_supply, nobles)
        flat_state = flatten_state(state)
        state_hashed = hasher.transform([flat_state]).toarray()

        done = False
        while not done:
            state_hashed_torch = torch.tensor(state_hashed, dtype=torch.float32).to(device)
            action = agent.act(state_hashed_torch, state['valid_actions'], state['all_actions'])
            next_state, reward, done, _ = env.step(action)
            flat_next_state = flatten_state(next_state)
            next_state_hashed = hasher.transform([flat_next_state]).toarray()

            next_state_hashed_torch = torch.tensor(next_state_hashed, dtype=torch.float32).to(device)
            agent.remember(state_hashed_torch, action, reward, next_state_hashed_torch, done)
            state = next_state
            state_hashed = next_state_hashed

        if len(agent.memory) > batch_size:
            if env.turns > 70:
                agent.replay(batch_size)
            else:
                agent.replay(batch_size)
        if e % 100 == 0:  # Evaluate the agent's performance on the test data every 10 episodes
            test_average_turns = test_agent(agent, device)
            trial.report(test_average_turns, e)  # Report the test performance to Optuna

            # Handle pruning based on the intermediate results
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return test_average_turns

if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Define the Optuna study and optimize the objective function
    study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
    study.optimize(lambda trial: train_agent(trial, device), n_trials=30)

    # Print the best hyperparameters
    print("Best hyperparameters:", study.best_params)


[I 2023-04-15 13:24:33,624] A new study created in memory with name: no-name-90305d3b-9790-4930-9370-137698fd6f5d
/usr/local/lib/python3.9/dist-packages/gym/spaces/box.py:85: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  self.dtype = np.dtype(dtype)
/usr/local/lib/python3.9/dist-packages/numpy/core/numeric.py:343: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  a = empty(shape, dtype, order)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:649: DeprecationWarning: Converting `np.integer` or `np.signedinteger` t

KeyboardInterrupt: ignored

### Training optimal model

In [ ]:
import torch

EPISODES = 10000
N_FEATURES = 100000
hasher = FeatureHasher(n_features=N_FEATURES, input_type='dict')

if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    state_size = N_FEATURES
    action_size = 67

    agent = DQNAgent(state_size, action_size, device) #runs in default mode.
    agent.gamma =  0.9138044631175565
    agent.epsilon = 0.8840360486793837
    agent.epsilon_min = 0.000010325101455740955
    agent.epsilon_decay = 0.9325720436508965
    agent.learning_rate = 0.00144458336962277

    batch_size = 32
    print('Parameters: discount: {}, initial epsilon:{},min_eps:{} ,decay:{},lr={}'.format(agent.gamma,agent.epsilon,agent.epsilon_min,agent.epsilon_decay,agent.learning_rate))

    training_cum_rewards = 0
    trainin_rewards = []
    training_turns_per_eps = []
    for e in range(EPISODES):
        state = env.reset(card_supply, nobles)
        flat_state = flatten_state(state)
        state_hashed = hasher.transform([flat_state]).toarray()

        done = False
        while not done:
            state_hashed_torch = torch.tensor(state_hashed, dtype=torch.float32).to(device)
            action = agent.act(state_hashed_torch, state['valid_actions'], state['all_actions'])
            next_state, reward, done, _ = env.step(action)
            flat_next_state = flatten_state(next_state)
            next_state_hashed = hasher.transform([flat_next_state]).toarray()

            next_state_hashed_torch = torch.tensor(next_state_hashed, dtype=torch.float32).to(device)
            agent.remember(state_hashed_torch, action, reward, next_state_hashed_torch, done)
            state = next_state
            state_hashed = next_state_hashed
            if done:
                print("episode: {}/{},turns: {} ,score: {}, e: {:.2}".format(e+1, EPISODES, env.turns, env.score, agent.epsilon))

        if len(agent.memory) > batch_size:
            if env.turns > 70:
                print('REPLAY FAILED GAME')
                agent.replay(batch_size)
            else:
                agent.replay(batch_size)
        # Optionally, save the agent weights after every 10 episodes
        if e % 10 == 0:
            agent.save("dqn_weights.pth")


Streaming output truncated to the last 5000 lines.
episode: 656/10000,turns: 41 ,score: 17, e: 1e-05
episode: 657/10000,turns: 61 ,score: 0, e: 1e-05
episode: 658/10000,turns: 61 ,score: 0, e: 1e-05
episode: 659/10000,turns: 41 ,score: 16, e: 1e-05
episode: 660/10000,turns: 41 ,score: 18, e: 1e-05
episode: 661/10000,turns: 61 ,score: 0, e: 1e-05
episode: 662/10000,turns: 61 ,score: 0, e: 1e-05
episode: 663/10000,turns: 41 ,score: 16, e: 1e-05
episode: 664/10000,turns: 61 ,score: 3, e: 1e-05
episode: 665/10000,turns: 41 ,score: 15, e: 1e-05
episode: 666/10000,turns: 61 ,score: 0, e: 1e-05
episode: 667/10000,turns: 41 ,score: 15, e: 1e-05
episode: 668/10000,turns: 36 ,score: 17, e: 1e-05
episode: 669/10000,turns: 61 ,score: 1, e: 1e-05
episode: 670/10000,turns: 40 ,score: 15, e: 1e-05
episode: 671/10000,turns: 41 ,score: 16, e: 1e-05
episode: 672/10000,turns: 38 ,score: 15, e: 1e-05
episode: 673/10000,turns: 42 ,score: 16, e: 1e-05
episode: 674/10000,turns: 42 ,score: 16, e: 1e-05
episod

In [ ]:
###TEST THE AGENT:
##100 episodes = 4min
##Create new env
turns = []
episodes = 1000
scores = []

for x in range(episodes):
    env = SplendorEnv(card_supply, nobles)
    # GET FIRST STATE
    state = env.reset(card_supply, nobles)
    ##flatten
    flat_state = flatten_state(state)
    ##Hash-it
    state_hashed = hasher.transform([flat_state]).toarray()
    done = False
    while not done:
        state_hashed_torch = torch.tensor(state_hashed, dtype=torch.float32).to(device)
        action = agent.act(state_hashed_torch, state['valid_actions'], state['all_actions'], train=False)
        next_state, reward, done, _ = env.step(action)
        flat_next_state = flatten_state(next_state)
        next_state_hashed = hasher.transform([flat_next_state]).toarray()
        next_state_hashed_torch = torch.tensor(next_state_hashed, dtype=torch.float32).to(device)

        state = next_state
        state_hashed = next_state_hashed

    turns.append(env.turns)
    print("turns: {}, score: {}, e: {}, episode: {}".format(env.turns, env.score, agent.epsilon, x))
    scores.append(env.score)

average_turns = sum(turns) / len(turns)
print("Average score over {} episodes: {:.2f}".format(episodes, average_turns))
losses = turns.count(61)  # Number of losses.
print('Number of Losses: {}'.format(losses))
filtered_scores = [score for score in turns if score != 61]
print('Average turns to win without losses:', sum(filtered_scores) / len(filtered_scores))
